<a href="https://colab.research.google.com/github/CodeByAtharva/Agentic-AI-Internship/blob/master/day%208/Universal_file_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q langchain-google-genai langchain-core tavily-python requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.6/426.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.7/233.7 kB 1.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.46.0 which is incompatible.


In [5]:
"""
Universal File Summarizer using LangChain
Supports: .txt, .pdf, .csv files
Generates 10-point bullet summaries using LLM
"""

# ========================================
# STEP 1: Install Required Packages
# ========================================
# Run this cell first in Google Colab
"""
!pip install langchain langchain-community langchain-openai pypdf python-dotenv -q
"""

# ========================================
# STEP 2: Import Libraries
# ========================================
import os
from google.colab import files
from langchain_community.document_loaders import TextLoader, PyPDFLoader, CSVLoader
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

# ========================================
# STEP 3: Set up OpenAI API Key
# ========================================
# Option 1: Direct input (for demo purposes)
from getpass import getpass
from google.colab import userdata
print("🔑 Please enter your OpenAI API Key:")

GEMINI_API = userdata.get('GeminiApi')


# Option 2: Use environment variable (if already set)
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# ========================================
# STEP 4: File Upload
# ========================================
print("\n" + "="*60)
print("📂 UNIVERSAL FILE SUMMARIZER")
print("="*60)
print("\nSupported formats: .txt, .pdf, .csv")
print("\nPlease upload your file...")

uploaded = files.upload()

if not uploaded:
    raise ValueError("❌ No file uploaded. Please upload a file.")

file_name = list(uploaded.keys())[0]
print(f"\n✅ File uploaded successfully: {file_name}")

# ========================================
# STEP 5: File Type Detection & Loading
# ========================================
print("\n" + "-"*60)
print("🔍 DETECTING FILE TYPE & LOADING CONTENT")
print("-"*60)

# Detect file type and use appropriate loader
if file_name.endswith(".txt"):
    print(f"📄 Detected: Plain Text File (.txt)")
    loader = TextLoader(file_name, encoding="utf-8")

elif file_name.endswith(".pdf"):
    print(f"📕 Detected: PDF Document (.pdf)")
    loader = PyPDFLoader(file_name)

elif file_name.endswith(".csv"):
    print(f"📊 Detected: CSV File (.csv)")
    loader = CSVLoader(file_name)

else:
    raise ValueError(f"❌ Unsupported file type: {file_name}\nPlease upload a .txt, .pdf, or .csv file.")

# Load documents
print(f"⏳ Loading content...")
docs = loader.load()

# ========================================
# STEP 6: Inspect Loaded Documents
# ========================================
print(f"\n✅ Content loaded successfully!")
print(f"\n📊 Document Analysis:")
print(f"   • Type: {type(docs)}")
print(f"   • Number of documents/pages: {len(docs)}")
print(f"   • Total characters: {sum(len(doc.page_content) for doc in docs):,}")

# Show preview of first document
preview_length = 300
preview = docs[0].page_content[:preview_length]
print(f"\n📋 Content Preview (first {preview_length} chars):")
print("-"*60)
print(preview)
if len(docs[0].page_content) > preview_length:
    print("...")
print("-"*60)

# ========================================
# STEP 7: Combine Content (if multiple pages)
# ========================================
# Combine all document content
combined_content = "\n\n".join([doc.page_content for doc in docs])

# Truncate if too long (to stay within token limits)
max_chars = 15000  # Adjust based on your needs
if len(combined_content) > max_chars:
    print(f"\n⚠️  Content is long ({len(combined_content):,} chars). Truncating to {max_chars:,} chars for summarization.")
    combined_content = combined_content[:max_chars] + "\n\n[Content truncated for summarization...]"

# ========================================
# STEP 8: Set up LLM and Prompt
# ========================================
print("\n" + "-"*60)
print("🤖 GENERATING SUMMARY WITH LLM")
print("-"*60)

# Initialize the LLM
llm = ChatGoogleGenerativeAI(
            model="gemini-2.5-flash-lite",
            temperature=0.7,
            google_api_key=GEMINI_API
        )

# Create prompt template
prompt_template = PromptTemplate(
    input_variables=["content"],
    template="""You are an expert summarizer. Analyze the following content and create a comprehensive 10-point bullet summary.

Each bullet point should:
- Capture a key insight, fact, or theme from the content
- Be clear, concise, and informative
- Be a complete sentence
- Cover different aspects of the content

Content to summarize:
{content}

Please provide exactly 10 bullet points that summarize the most important information:"""
)

# Create the chain
output_parser = StrOutputParser()
chain = prompt_template | llm | output_parser

# ========================================
# STEP 9: Generate Summary
# ========================================
print("⏳ Analyzing content and generating summary...")

try:
    summary = chain.invoke({"content": combined_content})

    # ========================================
    # STEP 10: Display Final Summary
    # ========================================
    print("\n" + "="*60)
    print("📝 SUMMARY GENERATED SUCCESSFULLY")
    print("="*60)
    print(f"\n📄 File: {file_name}")
    print(f"📊 Size: {len(combined_content):,} characters")
    print(f"\n{'='*60}")
    print("🎯 10-POINT SUMMARY")
    print(f"{'='*60}\n")
    print(summary)
    print(f"\n{'='*60}")
    print("✅ Summarization Complete!")
    print("="*60)

except Exception as e:
    print(f"\n❌ Error during summarization: {str(e)}")
    print("\nPlease check your API key and try again.")

# ========================================
# OPTIONAL: Save Summary to File
# ========================================
save_option = input("\n💾 Would you like to save the summary to a file? (yes/no): ").lower()

if save_option == "yes":
    output_filename = f"summary_{file_name.rsplit('.', 1)[0]}.txt"
    with open(output_filename, 'w', encoding='utf-8') as f:
        f.write(f"SUMMARY OF: {file_name}\n")
        f.write("="*60 + "\n\n")
        f.write(summary)
        f.write("\n\n" + "="*60)
        f.write(f"\nGenerated by Universal File Summarizer")

    print(f"✅ Summary saved to: {output_filename}")

    # Download the file
    files.download(output_filename)
    print("📥 File downloaded successfully!")

🔑 Please enter your OpenAI API Key:

📂 UNIVERSAL FILE SUMMARIZER

Supported formats: .txt, .pdf, .csv

Please upload your file...


Saving Atharva Full stack resume 2.pdf to Atharva Full stack resume 2.pdf

✅ File uploaded successfully: Atharva Full stack resume 2.pdf

------------------------------------------------------------
🔍 DETECTING FILE TYPE & LOADING CONTENT
------------------------------------------------------------
📕 Detected: PDF Document (.pdf)
⏳ Loading content...

✅ Content loaded successfully!

📊 Document Analysis:
   • Type: <class 'list'>
   • Number of documents/pages: 1
   • Total characters: 2,001

📋 Content Preview (first 300 chars):
------------------------------------------------------------
ATHARVA KAMBLE 
INFORMATION TECHNOLOGY 
atharva.kamble110586@gmail.com | (+91)9552851437 | LinkedIn | GitHub | Codolio 
 
CAREER OBJECTIVE 
Information Technology undergraduate seeking an internship in full -stack web development, aiming to apply coding 
skills and practical knowledge to build relia
...
------------------------------------------------------------

--------------------------------------

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 File downloaded successfully!


In [3]:
!pip install langchain langchain-community langchain-openai langchain-google-genai pypdf python-dotenv -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.46.0 which is incompatible.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
